In [93]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [94]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

,a,z,symbol,tf,datetime,time,open,high,low,close,tick_volume,spread,real_volume
0,1,-99998,WIN$,M1,2024-10-14 16:02:00,1728921720,140663.0,140716.0,140641.0,140700.0,5349,1,22765
1,2,-99997,WIN$,M1,2024-10-14 16:03:00,1728921780,140706.0,140706.0,140673.0,140689.0,2754,1,8644
2,3,-99996,WIN$,M1,2024-10-14 16:04:00,1728921840,140684.0,140706.0,140679.0,140684.0,2021,1,7869
3,4,-99995,WIN$,M1,2024-10-14 16:05:00,1728921900,140689.0,140727.0,140684.0,140722.0,2754,1,11333
4,5,-99994,WIN$,M1,2024-10-14 16:06:00,1728921960,140722.0,140722.0,140673.0,140684.0,2916,1,11662


In [95]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

,a,z,symbol,tf,datetime,time,open,high,low,close,tick_volume,spread,real_volume
0,1,-1244,WIN$,D1,2020-07-07,1594080000,147982.0,148853.0,146240.0,147111.0,3175030,1,11969548
1,2,-1243,WIN$,D1,2020-07-08,1594166400,147712.0,150453.0,147502.0,150415.0,3074071,1,11523486
2,3,-1242,WIN$,D1,2020-07-09,1594252800,150040.0,150768.0,148576.0,148838.0,3293543,1,11965440
3,4,-1241,WIN$,D1,2020-07-10,1594339200,148538.0,150566.0,148050.0,150040.0,2873888,1,10672868
4,5,-1240,WIN$,D1,2020-07-13,1594598400,151317.0,151752.0,147637.0,147637.0,3101361,1,11323137


# Data

In [96]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    Src['Datetime']                     = Query['datetime']
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if x >= 0 else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if x <  0 else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

,A,Z,Symbol,TF,Datetime,Ticks,Volume,Open,High,Low,Close,Price,Change,Hilo,Sign Bull,Sign Bear
0,1,-1244,WIN$,D1,2020-07-07,3175030,11969548,147982.0,148853.0,146240.0,147111.0,147111.0,-871.0,2613.0,NaN,-1.0
1,2,-1243,WIN$,D1,2020-07-08,3074071,11523486,147712.0,150453.0,147502.0,150415.0,150415.0,2703.0,2951.0,1.0,NaN
2,3,-1242,WIN$,D1,2020-07-09,3293543,11965440,150040.0,150768.0,148576.0,148838.0,148838.0,-1202.0,2192.0,NaN,-1.0
3,4,-1241,WIN$,D1,2020-07-10,2873888,10672868,148538.0,150566.0,148050.0,150040.0,150040.0,1502.0,2516.0,1.0,NaN
4,5,-1240,WIN$,D1,2020-07-13,3101361,11323137,151317.0,151752.0,147637.0,147637.0,147637.0,-3680.0,4115.0,NaN,-1.0


In [97]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']


    BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, 5200, 5500]
    BINS_HL  = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000]

    for x in BINS_CHG:  Calc[f'Chg Abs < {x}'] = (Calc['Chg Abs'] < x)
    for x in BINS_HL:   Calc[f'Hilo < {x}']    = (Calc['Hilo']    < x)


    FUTS = [1, 2]
     
    for x in FUTS:  Calc[f'Chg +{x}']       = Calc['Change'].shift(-x)
    for x in FUTS:  Calc[f'Chg Abs +{x}']   = Calc['Chg Abs'].shift(-x)
    for x in FUTS:  Calc[f'Chg Pos +{x}']   = Calc['Chg Pos'].shift(-x)
    for x in FUTS:  Calc[f'Chg Neg +{x}']   = Calc['Chg Neg'].shift(-x)

    for x in FUTS:  Calc[f'Hilo +{x}']      = Calc['Hilo'].shift(-x)
    for x in FUTS:  Calc[f'HL Pos +{x}']    = Calc['HL Pos'].shift(-x)
    for x in FUTS:  Calc[f'HL Neg +{x}']    = Calc['HL Neg'].shift(-x)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

,A,Z,Symbol,TF,Datetime,Ticks,Volume,Open,High,Low,Close,Price,Change,Hilo,Sign Bull,Sign Bear,Chg Abs,Chg Pos,Chg Neg,HL Pos,HL Neg,Chg Abs < 100,Chg Abs < 200,Chg Abs < 500,Chg Abs < 800,Chg Abs < 1000,Chg Abs < 1200,Chg Abs < 1500,Chg Abs < 1800,Chg Abs < 2000,Chg Abs < 2200,Chg Abs < 2500,Chg Abs < 2800,Chg Abs < 3000,Chg Abs < 3200,Chg Abs < 3500,Chg Abs < 3800,Chg Abs < 4000,Chg Abs < 4200,Chg Abs < 4500,Chg Abs < 4800,Chg Abs < 5000,Chg Abs < 5200,Chg Abs < 5500,Hilo < 1000,Hilo < 1500,Hilo < 2000,Hilo < 2500,Hilo < 3000,Hilo < 3500,Hilo < 4000,Hilo < 4500,Hilo < 5000,Hilo < 5500,Hilo < 6000,Hilo < 6500,Hilo < 7000,Hilo < 7500,Hilo < 8000,Chg +1,Chg +2,Chg Abs +1,Chg Abs +2,Chg Pos +1,Chg Pos +2,Chg Neg +1,Chg Neg +2,Hilo +1,Hilo +2,HL Pos +1,HL Pos +2,HL Neg +1,HL Neg +2
0,1,-1244,WIN$,D1,2020-07-07,3175030,11969548,147982.0,148853.0,146240.0,147111.0,147111.0,-871.0,2613.0,NaN,-1.0,871.0,NaN,-871.0,NaN,-2613.0,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,2703.0,-1202.0,2703.0,1202.0,2703.0,NaN,NaN,-1202.0,2951.0,2192.0,2951.0,NaN,NaN,-2192.0
1,2,-1243,WIN$,D1,2020-07-08,3074071,11523486,147712.0,150453.0,147502.0,150415.0,150415.0,2703.0,2951.0,1.0,NaN,2703.0,2703.0,NaN,2951.0,NaN,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,-1202.0,1502.0,1202.0,1502.0,NaN,1502.0,-1202.0,NaN,2192.0,2516.0,NaN,2516.0,-2192.0,NaN
2,3,-1242,WIN$,D1,2020-07-09,3293543,11965440,150040.0,150768.0,148576.0,148838.0,148838.0,-1202.0,2192.0,NaN,-1.0,1202.0,NaN,-1202.0,NaN,-2192.0,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,1502.0,-3680.0,1502.0,3680.0,1502.0,NaN,NaN,-3680.0,2516.0,4115.0,2516.0,NaN,NaN,-4115.0
3,4,-1241,WIN$,D1,2020-07-10,2873888,10672868,148538.0,150566.0,148050.0,150040.0,150040.0,1502.0,2516.0,1.0,NaN,1502.0,1502.0,NaN,2516.0,NaN,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,-3680.0,2170.0,3680.0,2170.0,NaN,2170.0,-3680.0,NaN,4115.0,3492.0,NaN,3492.0,-4115.0,NaN
4,5,-1240,WIN$,D1,2020-07-13,3101361,11323137,151317.0,151752.0,147637.0,147637.0,147637.0,-3680.0,4115.0,NaN,-1.0,3680.0,NaN,-3680.0,NaN,-4115.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,2170.0,360.0,2170.0,360.0,2170.0,360.0,NaN,NaN,3492.0,2230.0,3492.0,2230.0,NaN,NaN


# Stats

In [98]:
def MARKOV_CHAIN(Calc, when, _next):
    pipe = []
    for x in when: 
        Df = Calc[Calc[x]==1]

        for y in _next: 
            pipe.append({ 'Previous':x, 'Next':y, 
                'Count':          Df[y].count(), 
                'Avg':   np.round(Df[y].mean(),        0).astype(int),
                'Max':   np.round(Df[y].max(),         0).astype(int),
                'Q3':    np.round(Df[y].quantile(.75), 0).astype(int),
                'Med':   np.round(Df[y].median(),      0).astype(int),
                'Q1':    np.round(Df[y].quantile(.25), 0).astype(int),
                'Min':   np.round(Df[y].min(),         0).astype(int),
            })
        pass
    pass
    return pd.DataFrame(pipe)

In [99]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5500}'].columns, _next=['Chg Abs +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Chg Abs < 100,Chg Abs +1,61,1207,3536,1774,1030,333,74
1,Chg Abs < 200,Chg Abs +1,130,1247,5178,1691,1102,437,6
2,Chg Abs < 500,Chg Abs +1,307,1315,5678,1772,1100,550,6
3,Chg Abs < 800,Chg Abs +1,465,1331,5935,1840,1104,540,0
4,Chg Abs < 1000,Chg Abs +1,597,1308,5935,1804,1075,507,0
5,Chg Abs < 1200,Chg Abs +1,688,1286,5935,1800,1043,498,0
6,Chg Abs < 1500,Chg Abs +1,815,1311,5935,1829,1045,506,0
7,Chg Abs < 1800,Chg Abs +1,919,1301,5935,1836,1045,500,0
8,Chg Abs < 2000,Chg Abs +1,974,1304,5935,1831,1047,503,0
9,Chg Abs < 2200,Chg Abs +1,1015,1293,5935,1814,1045,496,0


In [100]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}':f'Chg Abs < {5500}'].columns, _next=['Hilo +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Chg Abs < 100,Hilo +1,61,2271,5258,2829,1989,1646,1006
1,Chg Abs < 200,Hilo +1,130,2406,7479,2874,2182,1663,1006
2,Chg Abs < 500,Hilo +1,307,2420,7479,2873,2160,1660,752
3,Chg Abs < 800,Hilo +1,465,2440,7479,2980,2215,1662,752
4,Chg Abs < 1000,Hilo +1,597,2418,7479,2902,2219,1660,655
5,Chg Abs < 1200,Hilo +1,688,2415,7479,2916,2216,1645,655
6,Chg Abs < 1500,Hilo +1,815,2447,7864,2983,2225,1661,593
7,Chg Abs < 1800,Hilo +1,919,2454,7864,2994,2240,1665,593
8,Chg Abs < 2000,Hilo +1,974,2447,7864,2988,2234,1660,593
9,Chg Abs < 2200,Hilo +1,1015,2441,7864,2977,2231,1658,593


In [101]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Hilo < {1000}':f'Hilo < {8000}'].columns, _next=['Chg Abs +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Hilo < 1000,Chg Abs +1,29,1051,3536,1475,1030,333,17
1,Hilo < 1500,Chg Abs +1,200,1039,5678,1488,908,327,0
2,Hilo < 2000,Chg Abs +1,466,1138,5678,1619,964,436,0
3,Hilo < 2500,Chg Abs +1,703,1189,5935,1666,980,446,0
4,Hilo < 3000,Chg Abs +1,894,1228,5935,1689,1000,472,0
5,Hilo < 3500,Chg Abs +1,1032,1251,5935,1730,1006,476,0
6,Hilo < 4000,Chg Abs +1,1101,1258,5935,1751,1006,480,0
7,Hilo < 4500,Chg Abs +1,1154,1287,5935,1816,1036,488,0
8,Hilo < 5000,Chg Abs +1,1195,1301,7641,1818,1043,491,0
9,Hilo < 5500,Chg Abs +1,1219,1311,7641,1828,1045,500,0


In [102]:
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Hilo < {1000}':f'Hilo < {8000}'].columns, _next=['Hilo +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Hilo < 1000,Hilo +1,29,1902,3969,2135,1833,1376,752
1,Hilo < 1500,Hilo +1,200,1929,5976,2301,1719,1379,655
2,Hilo < 2000,Hilo +1,466,2106,5976,2582,1900,1492,593
3,Hilo < 2500,Hilo +1,703,2207,7278,2664,2038,1550,593
4,Hilo < 3000,Hilo +1,894,2290,7278,2748,2109,1600,593
5,Hilo < 3500,Hilo +1,1032,2362,7731,2871,2190,1641,593
6,Hilo < 4000,Hilo +1,1101,2393,7864,2897,2211,1657,593
7,Hilo < 4500,Hilo +1,1154,2442,7864,2987,2242,1671,593
8,Hilo < 5000,Hilo +1,1195,2482,8487,3024,2259,1688,593
9,Hilo < 5500,Hilo +1,1219,2504,8487,3057,2289,1694,593


# Snippets

In [103]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if x >= 0 else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if x <  0 else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']

    BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, ...]
    BINS_HL  = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, ...] 

    for x in BINS_CHG:  Calc[f'Chg Abs < {x}'] = (Calc['Chg Abs'] < x)
    for x in BINS_HL:   Calc[f'Hilo < {x}']    = (Calc['Hilo']    < x)

    FUTS = [1, 2]
    for x in FUTS:  Calc[f'Chg +{x}']       = Calc['Change'].shift(-x)
    for x in FUTS:  Calc[f'Chg Abs +{x}']   = Calc['Chg Abs'].shift(-x)
    for x in FUTS:  Calc[f'Chg Pos +{x}']   = Calc['Chg Pos'].shift(-x)
    for x in FUTS:  Calc[f'Chg Neg +{x}']   = Calc['Chg Neg'].shift(-x)

    for x in FUTS:  Calc[f'Hilo +{x}']      = Calc['Hilo'].shift(-x)
    for x in FUTS:  Calc[f'HL Pos +{x}']    = Calc['HL Pos'].shift(-x)
    for x in FUTS:  Calc[f'HL Neg +{x}']    = Calc['HL Neg'].shift(-x)
    return Calc

In [104]:
def MARKOV_CHAIN(Calc, when, _next):
    pipe = []
    for x in when: 
        Df = Calc[Calc[x]==1]

        for y in _next: 
            pipe.append({ 'Previous':x, 'Next':y, 
                'Count':          Df[y].count(), 
                'Avg':   np.round(Df[y].mean(),        0).astype(int),
                'Max':   np.round(Df[y].max(),         0).astype(int),
                'Q3':    np.round(Df[y].quantile(.75), 0).astype(int),
                'Med':   np.round(Df[y].median(),      0).astype(int),
                'Q1':    np.round(Df[y].quantile(.25), 0).astype(int),
                'Min':   np.round(Df[y].min(),         0).astype(int),
            })
        pass
    pass
    return pd.DataFrame(pipe)


MARKOV_CHAIN(Calc, when=Calc.loc[:,    f'Hilo < {1000}':   f'Hilo < {8000}'].columns, _next=['Hilo +1'])
MARKOV_CHAIN(Calc, when=Calc.loc[:,    f'Hilo < {1000}':   f'Hilo < {8000}'].columns, _next=['Chg Abs +1'])
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}' :f'Chg Abs < {5500}'].columns, _next=['Hilo +1'])
MARKOV_CHAIN(Calc, when=Calc.loc[:, f'Chg Abs < {100}' :f'Chg Abs < {5500}'].columns, _next=['Chg Abs +1'])

,Previous,Next,Count,Avg,Max,Q3,Med,Q1,Min
0,Chg Abs < 100,Chg Abs +1,61,1207,3536,1774,1030,333,74
1,Chg Abs < 200,Chg Abs +1,130,1247,5178,1691,1102,437,6
2,Chg Abs < 500,Chg Abs +1,307,1315,5678,1772,1100,550,6
3,Chg Abs < 800,Chg Abs +1,465,1331,5935,1840,1104,540,0
4,Chg Abs < 1000,Chg Abs +1,597,1308,5935,1804,1075,507,0
5,Chg Abs < 1200,Chg Abs +1,688,1286,5935,1800,1043,498,0
6,Chg Abs < 1500,Chg Abs +1,815,1311,5935,1829,1045,506,0
7,Chg Abs < 1800,Chg Abs +1,919,1301,5935,1836,1045,500,0
8,Chg Abs < 2000,Chg Abs +1,974,1304,5935,1831,1047,503,0
9,Chg Abs < 2200,Chg Abs +1,1015,1293,5935,1814,1045,496,0
